In [1]:
from psychopy import visual, core, sound, event
import numpy as np
from scipy.io.wavfile import write
import random
from pylsl import StreamInfo, StreamOutlet, StreamInlet, resolve_stream
from threading import Thread
import csv
import asyncio
from idun_guardian_sdk import GuardianClient

pygame 2.6.0 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
subj = "001"

In [4]:
# Load the sound from the WAV file
sound_440Hz = sound.Sound("440Hz_tone.wav")
sound_587Hz = sound.Sound("587Hz_tone.wav")
# Play the sound
sound_587Hz.play()

In [5]:
device_address = 'E5:1E:FD:F5:15:26'
api_token = "idun_GAtJDPZJ1bbs47Mf4KEBA3-v35iudqE3NSGSLD3OE8zE8KN2CHcN809-"
client = GuardianClient(address=device_address, api_token=api_token)
RECORDING_TIMER: int = 60 * 60 * 2  # 2 hours

In [6]:
if __name__ == "__main__":

    info = StreamInfo("IDUN", "EEG", 1, 250, "float32", client.address)
    lsl_outlet = StreamOutlet(info, 20, 360)

    def lsl_stream_handler(event):
        message = event.message
        eeg = message["raw_eeg"]
        most_recent_ts = eeg[-1]["timestamp"]
        data = [sample["ch1"] for sample in eeg]
        lsl_outlet.push_chunk(data, most_recent_ts)

    client.subscribe_live_insights(
        raw_eeg=True,
        handler=lsl_stream_handler,
    )

    asyncio.run(client.start_recording(recording_timer=RECORDING_TIMER))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [9]:
async def main():
    info = StreamInfo("IDUN", "EEG", 1, 250, "float32", client.address)

if __name__ == "__main__":
    if not asyncio.get_event_loop().is_running():
        asyncio.run(main())
    else:
        asyncio.ensure_future(main())

    def lsl_stream_handler(event):
        message = event.message
        eeg = message["raw_eeg"]
        most_recent_ts = eeg[-1]["timestamp"]
        data = [sample["ch1"] for sample in eeg]
        lsl_outlet.push_chunk(data, most_recent_ts)

    client.subscribe_live_insights(
        raw_eeg=True,
        handler=lsl_stream_handler,
    )

    asyncio.run(client.start_recording(recording_timer=RECORDING_TIMER))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [6]:
# # Set up the LSL stream for markers
info = StreamInfo('MarkerStream', 'Markers', 1, 2, 'int32', 'myuidw43536')
outlet = StreamOutlet(info)

# Set up the window and fixation cross
win = visual.Window([800, 600], color='black')
fixation = visual.TextStim(win, text='+', color='white', height=0.1)
block_text = visual.TextStim(win, text='', color='white', height=0.1, pos=(0, 0))

# Resolve the EEG stream
print("Looking for an EEG stream...")
eeg_streams = resolve_stream('type', 'EEG')
eeg_inlet = StreamInlet(eeg_streams[0])

# # Resolve the Marker stream
print("Looking for a Marker stream...")
marker_streams = resolve_stream('type', 'Markers')
marker_inlet = StreamInlet(marker_streams[0])

# # Create lists to store the data
eeg_data = []
timestamps = []
markers = []
marker_timestamps = []

# def collect_eeg_data():
while True:
    eeg_sample, eeg_timestamp = eeg_inlet.pull_sample()
    eeg_data.append(eeg_sample)
    timestamps.append(eeg_timestamp)

        
eeg_thread = Thread(target=collect_eeg_data)
eeg_thread.daemon = True  # Ensures the thread will close when the main program exits
eeg_thread.start()
        
# Number of blocks and target sounds per block
num_blocks = 2
target_sound_count_per_block = 30


for block in range(num_blocks):
    # Display the block number
    block_text.text = f"Starting Block {block + 1}. Press Enter to start."
    block_text.draw()
    win.flip()

    # Wait for Enter key press to start the block
    while True:
        keys = event.waitKeys()
        if 'return' in keys:  # Check if Enter key (Return) is pressed
            break
        elif 'escape' in keys:  # Allow escape to quit before starting the block
            win.close()
            core.quit()

    fixation.draw()
    win.flip()
    
    # Counter for the target sound in each block
    target_sound_count = 0
    while target_sound_count < target_sound_count_per_block:
        # Check for escape key press
        if 'escape' in event.getKeys():
            win.close()
            core.quit()
        
        # Play the standard sound
        standard_sound_count = random.randint(7, 12)
        for _ in range(standard_sound_count):
            sound_440Hz.play()
            outlet.push_sample([1])  # Send LSL marker for 440 Hz sound
            markers.append(1)
            marker_timestamps.append(core.getTime())  # Store the time when the marker was sent
            
            core.wait(0.75)  # Wait for 0.75 second between sounds

            # Check for escape key press
            if 'escape' in event.getKeys():
                win.close()
                core.quit()

        # Play the target sound
        sound_587Hz.play()
        outlet.push_sample([2])  # Send LSL marker for 587 Hz sound
        target_sound_count += 1
        markers.append(2)
        marker_timestamps.append(core.getTime())  # Store the time when the marker was sent
        
        core.wait(0.75)  # Wait for 0.75 second before the next sound

    # Block completed
    print(f"Block {block + 1} completed")

# Close the window after all blocks are done
win.close()
core.quit()
 
    
print("Data collection finished.")

# Convert the lists to numpy arrays for further processing
eeg_data = np.array(eeg_data)            # Shape: (num_samples, num_channels)
timestamps = np.array(timestamps)        # Shape: (num_samples,)
markers = np.array(markers)              # Shape: (num_markers,)
marker_timestamps = np.array(marker_timestamps)  # Shape: (num_markers,)

NameError: name 'markers' is not defined

In [ ]:
# Save the arrays as CSV files
np.savetxt(f'BCI4Kids/data/IDUN_Pilot/eeg_data_{subj}A.csv', eeg_data, delimiter=',')
np.savetxt(f'BCI4Kids/data/IDUN_Pilot/timestamps_{subj}A.csv', timestamps, delimiter=',')
np.savetxt(f'BCI4Kids/data/IDUN_Pilot/markers_{subj}A.csv', markers, delimiter=',')
np.savetxt(f'BCI4Kids/data/IDUN_Pilot/marker_timestamps_{subj}A.csv', marker_timestamps, delimiter=',')

# OLD

In [3]:
# Parameters
sample_rate = 44100  # Hertz
duration = 0.1       # Seconds
frequency = 440      # Hertz

# Generate the sound
t = np.linspace(0, duration, int(sample_rate * duration), False)
tone = 0.5 * np.sin(2 * np.pi * frequency * t)

# Ensure it is in the correct format
tone = np.int16(tone * 32767)

# Write to a WAV file
write("440Hz_tone.wav", sample_rate, tone)


frequency = 587      # Hertz

# Generate the sound
t = np.linspace(0, duration, int(sample_rate * duration), False)
tone = 0.5 * np.sin(2 * np.pi * frequency * t)

# Ensure it is in the correct format
tone = np.int16(tone * 32767)

# Write to a WAV file
write("587Hz_tone.wav", sample_rate, tone)

In [5]:
# Set up the window and fixation cross
win = visual.Window([800, 600], color='black')
fixation = visual.TextStim(win, text='+', color='white', height=0.1)
block_text = visual.TextStim(win, text='', color='white', height=0.1, pos=(0, 0))



# Number of blocks and target sounds per block
num_blocks = 2
target_sound_count_per_block = 30



for block in range(num_blocks):
    # Display the block number
    block_text.text = f"Starting Block {block + 1}. Press Enter to start."
    block_text.draw()
    win.flip()
    
    # Wait for Enter key press to start the block
    while True:
        keys = event.waitKeys()
        if 'return' in keys:  # Check if Enter key (Return) is pressed
            break
        elif 'escape' in keys:  # Allow escape to quit before starting the block
            win.close()
            core.quit()

    fixation.draw()
    win.flip()
    
    # Counter for the target sound in each block
    target_sound_count = 0
    while target_sound_count < target_sound_count_per_block:
        # Check for escape key press
        if 'escape' in event.getKeys():
            win.close()
            core.quit()
        
        # Play the standard sound
        standard_sound_count = random.randint(7, 12)
        for _ in range(standard_sound_count):
            sound_440Hz.play()
            #outlet.push_sample([1])  # Send LSL marker for 440 Hz sound
            core.wait(0.75)  # Wait for 1 second between sounds

            # Check for escape key press
            if 'escape' in event.getKeys():
                win.close()
                core.quit()

        # Play the target sound
        sound_587Hz.play()
        #outlet.push_sample([2])  # Send LSL marker for 587 Hz sound
        target_sound_count += 1
        core.wait(0.75)  # Wait for 1 second before the next sound

    # Block completed
    print(f"Block {block + 1} completed")

# Close the window after all blocks are done
win.close()
core.quit()

9.8301 	WARNING 	Monitor specification not found. Creating a temporary one...


SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
try:
    label, s = streams.getStream(sampleRate=sampleRate, channels=channels, blockSize=blockSize)
    if label is None or s is None:
        raise ValueError("Failed to get a valid stream.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: name 'streams' is not defined


In [4]:
from psychopy import prefs
prefs.general['audioLib'] = ['sounddevice']

from psychopy import sound
sound_440Hz = sound.Sound(440, secs=0.1)

TypeError: cannot unpack non-iterable NoneType object